In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow_hub as hub
import tensorflow_datasets as tfds
from tensorflow.keras import layers

import numpy as np
import matplotlib.pyplot as plt
import tensorflow_hub as hub
import tensorflow_datasets as tfds
from tensorflow.keras import layers

In [ ]:
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow import keras
import cv2
import os
import numpy as np

In [ ]:
dataset_path= './top_15_species'

In [ ]:
def load_and_preprocess_images(dataset_path, image_size=(128, 128)):
    images = []
    labels = []

    # Iterate through class directories
    for class_dir in os.listdir(dataset_path):
        class_path = os.path.join(dataset_path, class_dir)
        class_label = int(class_dir)  # Convert directory name to label if needed

        # Iterate through images in the class directory
        for image_file in os.listdir(class_path):
            image_path = os.path.join(class_path, image_file)

            # Read and preprocess the image
            image = cv2.imread(image_path)
            image = cv2.resize(image, image_size)
            image = image.astype(np.float32) / 255.0  # Normalize pixel values

            # Append the image and label to lists
            images.append(image)
            labels.append(class_label)

    return np.array(images), np.array(labels)

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Constants
IMAGE_RES = 224
BATCH_SIZE = 32
# Load the dataset
data = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    image_size=(IMAGE_RES, IMAGE_RES),
    batch_size=BATCH_SIZE,
    validation_split=0.2,  # 20% of the data will be used for testing
    subset="training",     # Specify "training" to get the training subset
    seed=42,
    label_mode='int'
)
num_examples = data.cardinality().numpy()
num_classes = len(data.class_names)

# Split the data into training and testing
testing_data = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    image_size=(IMAGE_RES, IMAGE_RES),
    batch_size=BATCH_SIZE,
    validation_split=0.2,  # No validation split for testing
    seed=42,
    label_mode='int',
    subset="validation"    # Specify "validation" to get the testing subset
)

In [ ]:
for images, labels in data.take(1):  # Take one batch from the dataset
    print(labels)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(15, 10))

# Dictionary to store one image per species
class_images = {}
class_labels = {}

# Iterate through batches until all 15 species are found
for images, labels in train_batches:
    images = images.numpy()  # Convert tensors to NumPy
    labels = labels.numpy()

    for i in range(len(labels)):
        class_name = data.class_names[labels[i]]

        # Store only one image per class
        if class_name not in class_images:
            # Corrected the transpose to move color channel to last dimension
            class_images[class_name] = np.transpose(images[i] * 255, (1, 0, 2)).astype("uint8")
            class_labels[class_name] = labels[i]

        # Stop when we have all 15 species
        if len(class_images) == 15:
            break
    if len(class_images) == 15:
        break

# Plot the images with scale labels
for i, (class_name, img) in enumerate(class_images.items()):
    ax = plt.subplot(3, 5, i + 1)
    plt.imshow(img)
    plt.title(class_name)
    plt.axis("on")  # Turn on the axis

    # Label x and y axis
    ax.set_xlabel("Width (pixels)")
    ax.set_ylabel("Height (pixels)")

    # Set ticks at intervals (optional: adjust spacing)
    h, w, _ = img.shape
    ax.set_xticks(np.linspace(0, w, num=5))  # 5 tick marks along width
    ax.set_yticks(np.linspace(0, h, num=5))  # 5 tick marks along height

    # Format tick labels (optional)
    ax.set_xticklabels([f"{int(x)}" for x in np.linspace(0, w, num=5)])
    ax.set_yticklabels([f"{int(y)}" for y in np.linspace(0, h, num=5)])

plt.tight_layout()  # Adjust layout to prevent overlap
plt.show()


In [ ]:
base_model = tf.keras.applications.InceptionV3(
    input_shape=(IMAGE_RES, IMAGE_RES, 3),
    include_top=False,
    weights='imagenet'
)

In [ ]:
base_model.trainable = False

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(num_classes, activation='softmax')

# Create the final model
model = tf.keras.Sequential([
    base_model,
    global_average_layer,
    prediction_layer
])

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
history = model.fit(train_batches, validation_data=validation_batches, epochs=10)